#### [1] 데이터 준비
- 데이터 정리 및 두 데이터셋 병합

[1-1] 데이터 정리
<hr>

In [11]:
import pandas as pd

In [12]:
pd.read_csv('./Data/AI_made.csv')

,id,text,label
0,1,"대학교 3학년 재학 중, 지역 소상공인을 위한 온라인 주문·예약 웹 서비스를 팀 프...",0
1,2,현대자동차에 지원한 동기는 기술을 통해 사람의 이동 경험을 실질적으로 바꾸는 기업이...,0
2,3,대학교 2학년 재학 중 지역 소상공인을 돕는 마케팅 봉사 동아리에 참여하며 온라인 ...,0
3,4,대학교 재학 중 데이터 분석 프로젝트를 수행하며 숫자와 정보가 실제 의사결정에 영향...,0
4,5,저는 이베이코리아 플랫폼 내에서 판매되는 ‘멀티포트 USB 허브’와 같은 사람이라고...,0
...,...,...,...
77,78,지원한 직무 분야의 전문성을 키우기 위해 가장 꾸준히 노력했던 경험은 **이론과 실...,0
78,79,팀워크를 발휘해 공동 목표 달성에 기여한 경험은 대학 연합 봉사 동아리에서 프로젝트...,0
79,80,도전적인 목표를 세우고 끈질기게 노력했던 경험은 전공과 무관한 분야의 공모전에 도전...,0
80,81,저는 문제를 그냥 지나치지 않고 끝까지 정리하려는 사람입니다. 눈에 띄는 성과보다 ...,0


In [13]:
df = pd.read_csv("./Data/human_made.csv", encoding="utf-8")

# 필요한 컬럼만 남기기 (Unnamed 같은 쓰레기 컬럼 제거)
df = df[["id", "text", "label"]]

# 완전 빈 행 제거
df = df.dropna(subset=["id", "text", "label"])

# 공백만 있는 행 제거
df["text"] = df["text"].astype(str).str.strip()
df = df[df["text"] != ""]

# label 정수화
df["label"] = df["label"].astype(int)

df.to_csv("./Data/human_made_clean.csv", index=False, encoding="utf-8")


In [14]:

df = pd.read_csv("./Data/AI_made.csv", encoding="utf-8")

# 필요한 컬럼만 남기기 (Unnamed 같은 쓰레기 컬럼 제거)
df = df[["id", "text", "label"]]

# 완전 빈 행 제거
df = df.dropna(subset=["id", "text", "label"])

# 공백만 있는 행 제거
df["text"] = df["text"].astype(str).str.strip()
df = df[df["text"] != ""]

# label 정수화
df["label"] = df["label"].astype(int)

df.to_csv("./Data/AI_made_clean.csv", index=False, encoding="utf-8")


In [15]:
df = pd.read_csv('./Data/human_made_clean.csv', encoding='utf-8')
df.info()
print()
df = pd.read_csv('./Data/AI_made_clean.csv', encoding='utf-8')
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      81 non-null     int64
 1   text    81 non-null     str  
 2   label   81 non-null     int64
dtypes: int64(2), str(1)
memory usage: 2.0 KB

<class 'pandas.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      82 non-null     int64
 1   text    82 non-null     str  
 2   label   82 non-null     int64
dtypes: int64(2), str(1)
memory usage: 2.1 KB


[1-2] 데이터 병합 <hr>

In [16]:
# =========================
# 설정: 입력/출력 경로
# =========================
HUMAN_PATH = "./Data/human_made_clean.csv"
AI_PATH    = "./Data/AI_made_clean.csv"
OUT_PATH   = "./Data/all.csv"

# =========================
# 1) CSV 로드
# =========================
human = pd.read_csv(HUMAN_PATH, encoding="utf-8")  # utf-8로 저장했으면 utf-8로 바꿔도 됨
ai    = pd.read_csv(AI_PATH, encoding="utf-8")

# 컬럼 체크 (id, text, label 존재해야 함)
need_cols = {"id", "text", "label"}
if not need_cols.issubset(human.columns) or not need_cols.issubset(ai.columns):
    raise ValueError(f"CSV 컬럼 확인 필요: 반드시 {need_cols} 포함해야 함")

# =========================
# 2) 라벨 규칙 통일
#   목표: 0=Human, 1=AI
#   현재 파일은 반대로 들어가 있어서 뒤집어줌
# =========================
human["label"] = 0  # 사람 데이터는 무조건 0
ai["label"] = 1     # AI 데이터는 무조건 1

# (선택) 출처 컬럼 추가하면 나중에 확인하기 좋음
human["source"] = "human"
ai["source"] = "ai"

# =========================
# 3) 텍스트 정리 (최소만)
# =========================
def clean_text(s):
    s = str(s)
    s = s.replace("\ufeff", "")  # BOM 제거
    s = s.strip()
    return s

human["text"] = human["text"].apply(clean_text)
ai["text"] = ai["text"].apply(clean_text)

# 빈 텍스트 제거
human = human[human["text"] != ""]
ai = ai[ai["text"] != ""]

# (선택) 너무 짧은 텍스트 제거 (예: 20자 미만)
MIN_CHARS = 20
human = human[human["text"].str.len() >= MIN_CHARS]
ai = ai[ai["text"].str.len() >= MIN_CHARS]

# =========================
# 4) 병합 + 중복 제거
# =========================
all_df = pd.concat([human, ai], ignore_index=True)

# 텍스트 완전 동일 중복 제거(중요)
all_df = all_df.drop_duplicates(subset=["text"]).reset_index(drop=True)

# id가 겹칠 수 있으니 새 id 부여(권장)
all_df.insert(0, "new_id", range(1, len(all_df) + 1))

# =========================
# 5) 저장
# =========================
all_df.to_csv(OUT_PATH, index=False, encoding="utf-8")

print("✅ 저장 완료:", OUT_PATH)
print("전체 샘플 수:", len(all_df))
print("라벨 분포:\n", all_df["label"].value_counts())
print("샘플 컬럼:", list(all_df.columns))
print(all_df.head(3))

✅ 저장 완료: ./Data/all.csv
전체 샘플 수: 161
라벨 분포:
 label
1    82
0    79
Name: count, dtype: int64
샘플 컬럼: ['new_id', 'id', 'text', 'label', 'source']
   new_id  id                                               text  label source
0       1   1  너도 나도 창의성, 발상의 전환이 필요하다고 외칩니다. 하지만 주구장창 발상의 전환...      0  human
1       2   2  다양한 현장을 넘나들고 취재한 경험을 통해 통합적 인재로 한발 나아갈 수 있었습니다...      0  human
2       3   3  전략적 소통\n\n소통은 끊임없이 벌어집니다. 선의를 가진 사람과 소통할 경우엔 문...      0  human


In [1]:
import pandas as pd
import re

# 1. CSV 로드
df = pd.read_csv("./Data/all.csv")  
# 컬럼 이름이 다르면 여기서 수정
# df.columns -> 확인해봐도 좋아

# 2. 문장 분리 함수
def split_into_sentences(text):
    if pd.isna(text):
        return []
    
    # 문장 끝 기준 분리 (. ? !)
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    
    # 너무 짧은 문장 제거 (선택)
    sentences = [s for s in sentences if len(s) > 5]
    
    return sentences

# 3. 문단 → 문장 단위로 확장
rows = []

for _, row in df.iterrows():
    text = row["text"]
    label = row["label"]
    
    sentences = split_into_sentences(text)
    
    for sent in sentences:
        rows.append({
            "text": sent,
            "label": label
        })

# 4. 새로운 DataFrame
sentence_df = pd.DataFrame(rows)

# 5. 저장
sentence_df.to_csv("./Data/all_sentence.csv", index=False, encoding="utf-8-sig")

print("✅ 문장 단위 데이터 생성 완료")
print(sentence_df.head())


✅ 문장 단위 데이터 생성 완료
                                                text  label
0                     너도 나도 창의성, 발상의 전환이 필요하다고 외칩니다.      0
1                하지만 주구장창 발상의 전환에만 매몰되는 것이 혁신이 아닙니다.      0
2                         가장 중요한 것은 기본에 충실히 하는 것입니다.      0
3  탄탄한 기본이 뒷받침된 상태에서 적재적소에 발상의 전환이 가미될 때 남들과 차별화된...      0
4        결과가 뒤따르지 않는 창의성, 혁신은 잠깐의 일탈일 뿐 인정받을 수 없습니다.      0
